In [1]:
import os

import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2 as cv
import json

In [2]:
from sequential.model import LSTMActionClassifier
from non_sequential.model import MLP
from combine import Ensemble

In [3]:
non_seq_model = MLP(816, 8)

feature_vector_size = 1000
hidden_size = 512
num_frames = 3
num_classes = 8
seq_model = LSTMActionClassifier(feature_vector_size,
                             num_frames,
                             hidden_size,
                             num_classes)

In [4]:
state_dict_non_seq = torch.load("non_seq_model_params.pth")
state_dict_seq = torch.load("seq_model_params.pth")

non_seq_model.load_state_dict(state_dict_non_seq)
seq_model.load_state_dict(state_dict_seq)

<All keys matched successfully>

In [5]:
ensemble = Ensemble(seq_model, non_seq_model)

In [6]:
root_dir = os.path.join(".", "test data")
# manually, since yolo integration is not done yet
data = [
    {
        "file_name": "00041.jpg",
        "bb_info": [
            [515, 121, 135, 147],
            [573, 184, 278, 292],
            [203, 170, 303, 164]
        ],
        "object_classes": [5, 3, 0]
    },
    {
        "file_name": "00043.jpg",
        "bb_info": [
            [542, 131, 218, 254],
            [605, 226, 240, 253],
            [336, 121, 277, 67]
        ],
        "object_classes": [5, 3, 0]
    },
    {
        "file_name": "00045.jpg",
        "bb_info": [
            [577, 134, 213, 262],
            [624, 217, 226, 258],
            [497, 33, 305, 58]
        ],
        "object_classes": [5, 3, 0]
    },
]

In [7]:
predictions = ensemble.predict(root_dir, data, prob=True)

c:\Users\Marc\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [8]:
predictions

{'non-sequential': [[0.1795438975095749,
   0.4835582375526428,
   0.06574470549821854,
   0.11743192374706268,
   0.041275620460510254,
   0.033652953803539276,
   0.06021096929907799,
   0.018581723794341087],
  [0.07670272141695023,
   0.8471351265907288,
   0.017162462696433067,
   0.024682795628905296,
   0.007405831012874842,
   0.00550815649330616,
   0.018550200387835503,
   0.002852613804861903],
  [0.08361814171075821,
   0.8744136691093445,
   0.002211113926023245,
   0.006557185668498278,
   0.0035323507618159056,
   0.009885214269161224,
   0.017233651131391525,
   0.002548637567088008]],
 'sequential': [[0.1389925330877304,
   0.14040105044841766,
   0.11065858602523804,
   0.14159323275089264,
   0.11011433601379395,
   0.11361302435398102,
   0.14515472948551178,
   0.09947250783443451]]}

In [9]:
ensemble.vote()

'assemble leg'